# Detectron2 inference


## Installing Dependencies

In [ ]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-t3r5ptq7
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-t3r5ptq7
1.5.0+cu101 False
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
# install detectron2:
!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html


## Downloading the training set

In [ ]:
!curl -L "https://app.roboflow.com/ds/Vo82n1pI0h?key=pm0pSoDvYq" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   887  100   887    0     0   1537      0 --:--:-- --:--:-- --:--:--  1534
100 41.7M  100 41.7M    0     0  34.5M      0  0:00:01  0:00:01 --:--:--  274M
Archive:  roboflow.zip
 extracting: README.roboflow.txt     
   creating: test/
 extracting: test/OC_paper_10page_19_GDOg08g_jpg.rf.e354ae0ba8ae7a0a5d2f967ef2cc9e1e.jpg  
 extracting: test/OC_paper_10page_22_3PkZj5o_jpg.rf.64b6d376b06de83cc850cdb01d9ee9c6.jpg  
 extracting: test/OC_paper_10page_52_1vwFCFL_jpg.rf.465125e91f26498a9570dad3a57dabfc.jpg  
 extracting: test/OC_paper_10page_57_sqn1Vg0_jpg.rf.de26c677afdf4d72edeabf8f7f8c52cd.jpg  
 extracting: test/OC_paper_10page_60_lry8YBM_jpg.rf.e9500b074524ca78911fb575dcc827d4.jpg  
 extracting: test/OC_paper_10page_82_HGZYakz_jpg.rf.804b7e8e3c73c3581c42d52fda630e81.jpg  
 extracting: test/OC_paper_10page_83_bDSItvF_jpg.rf.c58bd1

`cv2.imshow()` function isn't supported on colab, so `cv2_imshow` needs to be imported instead

In [ ]:
from google.colab.patches import cv2_imshow

## Defining the Detector Class with the required functions

In [ ]:
from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import ColorMode, Visualizer
# from detectron2 import model_zoo

import cv2
import  numpy as np
import os


class Detector:
    def __init__(self):
        self.cfg = get_cfg()

        # Load config and model
        self.cfg.merge_from_file('/content/config.yaml')
        # #self.cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url('COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml')
        # self.cfg.MODEL.WEIGHTS = './model_final.pth'

        self.cfg.MODEL.WEIGHTS = '/content/model_final.pth'
        self.cfg.DATASETS.TEST = ("my_dataset_test", )
        self.cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8   # set the testing threshold for this model

        self.cfg.MODEL.DEVICE = "cpu" # Change to cpu while deploying

        self.predictor = DefaultPredictor(self.cfg)
        MetadataCatalog.get(self.cfg.DATASETS.TRAIN[0]).thing_classes = ['cells', 'intermediates', 'markush-structures', 'reactions', 'relevant-structures', 'substitues']
        self.metadata = MetadataCatalog.get(self.cfg.DATASETS.TRAIN[0])

    def onImage(self, imagePath, saveDir, fileName, save_crop=False):
        image = cv2.imread(imagePath)
        predictions = self.predictor(image)

        viz = Visualizer(image[:,:,::-1], metadata = self.metadata, scale = 1)        
        output = viz.draw_instance_predictions(predictions["instances"].to("cpu"))
        save_path = saveDir + '/' + fileName
        cv2.imwrite(save_path, output.get_image()[:,:,::-1])
        # For debugging on colab
        # cv2_imshow(output.get_image()[:,:,::-1])
        
        if save_crop:
            image = image[:,:,::-1]
            boxes = predictions['instances'].pred_boxes
            classes = predictions['instances'].pred_classes
            # print(classes)
            i=0
            for box in boxes:
                x1, y1, x2, y2 = int(box[0]), int(box[1]), int(box[2]), int(box[3])
                cropped = image[y1:y2, x1:x2]
                detected_class_index = classes[i].item()
                detected_class = MetadataCatalog.get(self.cfg.DATASETS.TRAIN[0]).thing_classes[detected_class_index]
                # cv2_imshow(cropped)
                # print(detected_class)
                
                crop_save_dir = saveDir+'/'+detected_class
                
                if not os.path.exists(crop_save_dir):
                    os.mkdir(crop_save_dir)
                crop_path = crop_save_dir+'/'+fileName+'_'+str(i)+'.jpg'
                cv2.imwrite(crop_path, cropped)
                

                # print(len(predictions['instances'].pred_boxes))

def detectron2_inference(images_dir, save_dir, detector_):
    for file in os.listdir(images_dir):
        if file.endswith(".jpg"):
            image_name = file[:-4]
            detector_.onImage(
                fileName = file,
                imagePath = (images_dir+'/'+file),
                saveDir=save_dir,
                save_crop=True
            )

# Testing it
**Make sure the config file and weights file are uploaded**

In [ ]:
!sudo rm -r /content/results

detector = Detector()
save_dir = '/content/results'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

detectron2_inference(
    detector_ = detector,
    images_dir = '/content/valid',
    save_dir = (save_dir)

)